In [245]:

#------------------------------------------------------------------------
# WES 269 SBUS Demo Code
#
# This code serves as a baseline for configuring the SPI core
#
#------------------------------------------------------------------------

from pynq import Overlay
from pynq import MMIO
import time
import numpy as np

In [246]:

# Load bitstream
ol = Overlay('SPI_PMODNAV_test4.bit')

# Download bitstream to FPGA
t_before_bitstream = time.time()
ol.download()
t_after_bitstream = time.time()
print('  ')
print(t_after_bitstream - t_before_bitstream, 'seconds to program bitstream')
ol.ip_dict

  
0.4123663902282715 seconds to program bitstream


{'axi_quad_spi_0': {'addr_range': 65536,
  'driver': pynq.overlay.DefaultIP,
  'fullpath': 'axi_quad_spi_0',
  'gpio': {},
  'interrupts': {},
  'phys_addr': 1105199104,
  'state': None,
  'type': 'xilinx.com:ip:axi_quad_spi:3.2'}}

In [247]:
!cat /proc/cpuinfo

processor	: 0
model name	: ARMv7 Processor rev 0 (v7l)
BogoMIPS	: 650.00
Features	: half thumb fastmult vfp edsp neon vfpv3 tls vfpd32 
CPU implementer	: 0x41
CPU architecture: 7
CPU variant	: 0x3
CPU part	: 0xc09
CPU revision	: 0

processor	: 1
model name	: ARMv7 Processor rev 0 (v7l)
BogoMIPS	: 650.00
Features	: half thumb fastmult vfp edsp neon vfpv3 tls vfpd32 
CPU implementer	: 0x41
CPU architecture: 7
CPU variant	: 0x3
CPU part	: 0xc09
CPU revision	: 0

Hardware	: Xilinx Zynq Platform
Revision	: 0003
Serial		: 0000000000000000


In [248]:
# testing SPI output

# SPI CORE config
IP_BASE_ADDRESS = 0x41E00000
ADDRESS_RANGE = 0xFFFF


# Address space definitions for PMODNAV
ACC_X_L_ADDR   =  0x28

MAG_X_L_ADDR   =  0x28

GYRO_X_L_ADDR  =  0x18
GYRO_Y_L_ADDR  =  0x1A
GYRO_Z_L_ADDR  =  0x1C

ALT_X_L_ADDR   =  0x28

CTRL_REG6_XL_ADDR  =  0x20

X_DIR_SEL    =   0x01
Y_DIR_SEL    =   0x02
Z_DIR_SEL    =   0x03

READ_CFG    =   0x80  # bit 7 high
WRITE_CFG   =   0x7F  # bit 7 low
MS_CFG      =   0xC0  # 11000000

STATE_ON    =   0x01
STATE_OFF   =   0x00

CTRL_REG5_XL =  0x1F
CTRL_REG6_XL =  0x20
CTRL_REG4    =  0x1E

CTRL_REG1_M  =  0x20
CTRL_REG2_M  =  0x21
CTRL_REG3_M  =  0x22
CTRL_REG4_M  =  0x23
CTRL_REG5_M  =  0x24

CTRL_REG1_G  =  0x10
CTRL_REG2_G  =  0x11
CTRL_REG3_G  =  0x12

CTRL_REG1_ALT = 0x20
CTRL_REG2_ALT = 0x21
CTRL_REG4_ALT = 0x23

GYRO_WHO_AM_I  = 0x0F
# end PMODNav definitions

# creating SPI CORE object for messages
#set third argument to true if want debug
SPI_CORE = MMIO(IP_BASE_ADDRESS,ADDRESS_RANGE, False)

# this is a basic configuration of the SPI core
# I would read through the docs a bit more and make sure this is correct
# it seems to work as is, 8 bit addres and 8 bit data with SS and CLK

# testing config
SPI_CORE.write(0x60, 0x006) # enable SPI core in master mode, auto SS
SPI_CORE.write(0x70, 0xFFFE) # enable SS 0
read1 = SPI_CORE.read(0x60)
read2 = SPI_CORE.read(0x70)

print(read1)
print(read2)
print('done')

6
0
done


### Testing write to SPI Core

In [249]:
WRITE_OFFSET = 0x68
ADDRESS = 0x66 # made up address, 01100110
DATA = 0xD5
READ_OFFSET = 0x6C

# I put this together for a basic configuration of the messages
# each read/write needs to be tailored so
MESSAGE = (ADDRESS << 8) | DATA 


SPI_CORE.write(WRITE_OFFSET, MESSAGE) # try to send data
print('done')

done


### Testing read from SPI Core

In [250]:
r1 = SPI_CORE.read(READ_OFFSET)

print('offset: {0:x}\t'.format(READ_OFFSET) + 'data: {0:x}'.format(r1))

offset: 6c	data: 0


### Testing SPI with Pmod Nav

In [251]:
WHO_AM_I = 0x8F # address for who_am_i register from Nav, expect to read back 0x68
DATA_NAV = 0x55 # sending random data
MESSAGE_NAV = (WHO_AM_I << 8) | DATA_NAV
# expect to read a 0x68 based on pg. 45 of lhttps://www.st.com/resource/en/datasheet/lsm9ds1.pdf
SPI_CORE.write(WRITE_OFFSET, MESSAGE_NAV) # try to send data

In [252]:
r_nav= SPI_CORE.read(READ_OFFSET)
print('offset: {0:x}\t'.format(READ_OFFSET) + 'data: {0:x}'.format(r_nav))

offset: 6c	data: 68


### Testing Nav Driver Code

The methods below are a step closer for creating the Custom Nav Driver Code to function with the SPI Core.

In [253]:
# Custom PMODNav Driver code
NO_DATA = 0x00

def Nav_SPI_Write(addr, val):
    tx_byte = addr & WRITE_CFG
    #tx_byte = addr
    message = (tx_byte << 8) | val
    SPI_CORE.write(WRITE_OFFSET,message)

def Nav_SPI_Read(addr):
    tx_byte = addr | READ_CFG
    val = NO_DATA # don't think data needs to be sent when reading
    message = (tx_byte << 8) | val
    SPI_CORE.write(WRITE_OFFSET,message)
    r = SPI_CORE.read(READ_OFFSET)
    
    return r

def Nav_SPI_Acc_Gyro_pwr(state):
    if(state==STATE_ON):
        print("--ACC and GYRO Power ON--\r\n")
        Nav_SPI_Write(CTRL_REG5_XL, 0x38)
        Nav_SPI_Write(CTRL_REG6_XL, 0xC0)
        Nav_SPI_Write(CTRL_REG1_G, 0xC0)
        Nav_SPI_Write(CTRL_REG4, 0x38)
    else:
        print("--ACC and GYRO Power OFF")
        Nav_SPI_Write(CTRL_REG5_XL, 0x00)
        Nav_SPI_Write(CTRL_REG6_XL, 0x00)
        Nav_SPI_Write(CTRL_REG4, 0x00)
        Nav_SPI_Write(CTRL_REG1_G, 0x00)
    
    return 0

def Nav_SPI_Gyro_GetData(axis):
    if(axis == X_DIR_SEL):
        return Nav_SPI_Read(GYRO_X_L_ADDR)
    elif(axis == Y_DIR_SEL):
        return Nav_SPI_Read(GYRO_Y_L_ADDR)
    else:
        return Nav_SPI_Read(GYRO_Z_L_ADDR)

In [254]:
pwr = Nav_SPI_Acc_Gyro_pwr(1)

--ACC and GYRO Power ON--



### Collecting gyro single samples

In [258]:
gyro_data_x = Nav_SPI_Gyro_GetData(X_DIR_SEL)
print('gyro raw data x is: \n', gyro_data_x)
#time.sleep(2)
gyro_data_y = Nav_SPI_Gyro_GetData(Y_DIR_SEL)
print('gyro raw data y is: \n', gyro_data_y)
#time.sleep(2)
gyro_data_z = Nav_SPI_Gyro_GetData(Z_DIR_SEL)
print('gyro raw data z is: \n', gyro_data_z)

gyro raw data x is: 
 235
gyro raw data y is: 
 91
gyro raw data z is: 
 90


### Collecting gyro multiple samples

In [260]:
GYRO_RAW = list()

T_SAMPLE = 30
FS = 2
N_SAMPLE = T_SAMPLE*FS

for x in range(0, N_SAMPLE):
    gyro_data_x = Nav_SPI_Gyro_GetData(X_DIR_SEL)
    gyro_data_y = Nav_SPI_Gyro_GetData(Y_DIR_SEL)
    gyro_data_z = Nav_SPI_Gyro_GetData(Z_DIR_SEL)

    GYRO_RAW.append(gyro_data_x)
    GYRO_RAW.append(gyro_data_y)
    GYRO_RAW.append(gyro_data_z)
    
    print("GYRO_RAW:", GYRO_RAW[(x*3) + 0], "\t", GYRO_RAW[(x*3) + 1], "\t", GYRO_RAW[(x*3) + 2], "\r\n")
    #print("GYRO_DPS:", GYRO_DPS[(x*3) + 0], "\t", GYRO_DPS[(x*3) + 1], "\t", GYRO_DPS[(x*3) + 2], "\r\n")
    
    time.sleep(1/FS)

GYRO_RAW: 235 	 72 	 154 

GYRO_RAW: 231 	 100 	 94 

GYRO_RAW: 208 	 142 	 80 

GYRO_RAW: 208 	 114 	 94 

GYRO_RAW: 215 	 77 	 127 

GYRO_RAW: 217 	 95 	 129 

GYRO_RAW: 207 	 90 	 100 

GYRO_RAW: 207 	 88 	 84 

GYRO_RAW: 224 	 115 	 102 

GYRO_RAW: 200 	 120 	 136 

GYRO_RAW: 196 	 92 	 82 

GYRO_RAW: 194 	 121 	 127 

GYRO_RAW: 210 	 111 	 109 

GYRO_RAW: 219 	 92 	 118 

GYRO_RAW: 212 	 99 	 101 

GYRO_RAW: 227 	 97 	 143 

GYRO_RAW: 205 	 103 	 101 

GYRO_RAW: 209 	 85 	 133 

GYRO_RAW: 181 	 98 	 92 

GYRO_RAW: 212 	 97 	 123 

GYRO_RAW: 212 	 104 	 113 

GYRO_RAW: 221 	 107 	 117 

GYRO_RAW: 213 	 114 	 114 

GYRO_RAW: 196 	 95 	 97 

GYRO_RAW: 219 	 71 	 107 

GYRO_RAW: 177 	 108 	 89 

GYRO_RAW: 217 	 91 	 105 

GYRO_RAW: 212 	 108 	 139 

GYRO_RAW: 200 	 91 	 99 

GYRO_RAW: 215 	 110 	 104 

GYRO_RAW: 191 	 62 	 129 

GYRO_RAW: 179 	 139 	 130 

GYRO_RAW: 204 	 85 	 119 

GYRO_RAW: 218 	 114 	 98 

GYRO_RAW: 211 	 67 	 98 

GYRO_RAW: 210 	 116 	 84 

GYRO_RAW: 209 	 86 	 10